In [1]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.2 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import roc_auc_score, make_scorer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

# Import models for comparison
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import (
    RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier,
    HistGradientBoostingClassifier
)
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

# Load data
def load_data(train_path, test_path):
    train = pd.read_csv(train_path)
    test = pd.read_csv(test_path)
    return train, test

# Separate features and target variable
def preprocess_data(train, test, target_column='DiagPeriodL90D'):
    X = train.drop(columns=[target_column, 'patient_id'])
    y = train[target_column]
    X_test = test.drop(columns=['patient_id'])
    return X, y, X_test

# Define preprocessors
def create_preprocessor(X):
    numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
    categorical_features = X.select_dtypes(include=['object']).columns.tolist()

    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OneHotEncoder(handle_unknown='ignore'))
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)
        ]
    )
    return preprocessor

# Initialize models
def initialize_models():
    return {
        'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
        'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
        'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
        'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
        'CatBoost': CatBoostClassifier(verbose=0, random_state=42),
        'LightGBM': LGBMClassifier(random_state=42),
        'SVC': SVC(probability=True, random_state=42),
        'K-Nearest Neighbors': KNeighborsClassifier(),
        'GaussianNB': GaussianNB(),
        'Extra Trees': ExtraTreesClassifier(n_estimators=100, random_state=42),
        'HistGradientBoosting': HistGradientBoostingClassifier(random_state=42),
        'Voting Classifier': VotingClassifier(estimators=[
            ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
            ('gb', GradientBoostingClassifier(n_estimators=100, random_state=42)),
            ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)),
            ('cat', CatBoostClassifier(verbose=0, random_state=42)),
            ('lgb', LGBMClassifier(random_state=42))
        ], voting='soft')
    }

# Evaluate models
def evaluate_models(models, preprocessor, X, y, n_splits=5):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    scorer = make_scorer(roc_auc_score, needs_proba=True)
    results = {}

    for name, model in models.items():
        pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', model)])
        cv_scores = cross_val_score(pipeline, X, y, cv=skf, scoring=scorer, n_jobs=-1)
        results[name] = {
            'Mean ROC-AUC': cv_scores.mean(),
            'Std Dev': cv_scores.std(),
            'Scores': cv_scores
        }
        print(f"{name}: Mean ROC-AUC = {cv_scores.mean():.4f}, Std = {cv_scores.std():.4f}")
    return pd.DataFrame(results).T

# Train best model on full data and make predictions
def train_best_model_and_predict(best_model_name, models, preprocessor, X, y, X_test, test_data):
    best_model = models[best_model_name]
    final_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', best_model)])
    final_pipeline.fit(X, y)

    predictions = final_pipeline.predict_proba(X_test)[:, 1]  # Probability of DiagPeriodL90D=1
    submission = pd.DataFrame({
        'patient_id': test_data['patient_id'],
        'DiagPeriodL90D': predictions
    })
    submission.to_csv('submission.csv', index=False)
    print(f"Submission file created as 'submission.csv' with best model: {best_model_name}")

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [3]:
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.ensemble import HistGradientBoostingClassifier  # Ensuring this is imported if used

# Custom transformer to convert sparse matrix to dense matrix
class SparseToDenseTransformer(TransformerMixin, BaseEstimator):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X.toarray() if hasattr(X, 'toarray') else X

# Function to check if model needs dense data
def needs_dense_data(model):
    # Add models that require dense data here
    return isinstance(model, (GaussianNB, HistGradientBoostingClassifier))

# Adjusted evaluate_models function
def evaluate_models(models, preprocessor, X, y, n_splits=5):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    scorer = make_scorer(roc_auc_score, needs_proba=True)
    results = {}

    for name, model in models.items():
        if needs_dense_data(model):
            # Wrap models needing dense data with SparseToDenseTransformer
            pipeline = Pipeline(steps=[
                ('preprocessor', preprocessor),
                ('to_dense', SparseToDenseTransformer()),
                ('classifier', model)
            ])
        else:
            # Standard pipeline without conversion for other models
            pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', model)])

        # Evaluate using cross-validation
        cv_scores = cross_val_score(pipeline, X, y, cv=skf, scoring=scorer, n_jobs=-1)

        results[name] = {
            'Mean ROC-AUC': cv_scores.mean(),
            'Std Dev': cv_scores.std(),
            'Scores': cv_scores
        }
        print(f"{name}: Mean ROC-AUC = {cv_scores.mean():.4f}, Std = {cv_scores.std():.4f}")

    return pd.DataFrame(results).T

In [4]:
if __name__ == "__main__":
    # Paths to data files
    train_path = 'processed_train_data.csv'
    test_path = 'processed_test_data.csv'

    # Load datasets
    train_data, test_data = load_data(train_path, test_path)

    # Preprocess data
    X, y, X_test = preprocess_data(train_data, test_data)

    # Create preprocessor
    preprocessor = create_preprocessor(X)

    # Initialize models
    models = initialize_models()

    # Evaluate models and get results
    model_results = evaluate_models(models, preprocessor, X, y)

    # Select the best model by highest ROC-AUC mean
    best_model_name = model_results['Mean ROC-AUC'].idxmax()
    print(f"\nBest model: {best_model_name} with Mean ROC-AUC: {model_results['Mean ROC-AUC'].max():.4f}")

    # Train the best model and predict on test set
    train_best_model_and_predict(best_model_name, models, preprocessor, X, y, X_test, test_data)


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py:610: FutureWarning: The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid for Gradient Boosting
param_grid = {
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__max_depth': [3, 5, 7],
    'classifier__learning_rate': [0.01, 0.1, 1.0]
}

# Create a GradientBoostingClassifier instance
gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)

# Create a pipeline with the preprocessor and GradientBoostingClassifier
gb_pipeline = Pipeline([('preprocessor', preprocessor), ('classifier', gb_model)])

# Perform GridSearchCV to find the best hyperparameters
grid_search = GridSearchCV(gb_pipeline, param_grid, scoring='roc_auc', cv=5, n_jobs=-1)
grid_search.fit(X, y)

# Print the results for each hyperparameter combination
print("Gradient Boosting Hyperparameter Tuning Results:")
results = pd.DataFrame(grid_search.cv_results_)
print(results[['params', 'mean_test_score', 'std_test_score']])


# Get the best hyperparameters and score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f"\nBest Hyperparameters: {best_params}")
print(f"Best Mean ROC-AUC Score: {best_score:.4f}")

Gradient Boosting Hyperparameter Tuning Results:
                                               params  mean_test_score  \
0   {'classifier__learning_rate': 0.01, 'classifie...         0.758230   
1   {'classifier__learning_rate': 0.01, 'classifie...         0.758230   
2   {'classifier__learning_rate': 0.01, 'classifie...         0.758236   
3   {'classifier__learning_rate': 0.01, 'classifie...         0.768243   
4   {'classifier__learning_rate': 0.01, 'classifie...         0.768254   
5   {'classifier__learning_rate': 0.01, 'classifie...         0.768250   
6   {'classifier__learning_rate': 0.01, 'classifie...         0.786525   
7   {'classifier__learning_rate': 0.01, 'classifie...         0.786712   
8   {'classifier__learning_rate': 0.01, 'classifie...         0.787032   
9   {'classifier__learning_rate': 0.1, 'classifier...         0.799470   
10  {'classifier__learning_rate': 0.1, 'classifier...         0.799276   
11  {'classifier__learning_rate': 0.1, 'classifier...         0

In [ ]:

# Define the parameter grid for CatBoost
param_grid_catboost = {
    'classifier__iterations': [100, 200],
    'classifier__learning_rate': [0.01, 0.1],
    'classifier__depth': [4, 6],
    'classifier__random_strength': [0.5, 1.0],  # Added random_strength
    'classifier__bagging_temperature': [0.0, 0.5]  # Added bagging_temperature
}

# Create a CatBoostClassifier instance
catboost_model = CatBoostClassifier(verbose=0, random_state=42)

# Create a pipeline with the preprocessor and CatBoostClassifier
catboost_pipeline = Pipeline([('preprocessor', preprocessor), ('classifier', catboost_model)])

# Perform GridSearchCV to find the best hyperparameters
grid_search_catboost = GridSearchCV(catboost_pipeline, param_grid_catboost, scoring='roc_auc', cv=5, n_jobs=-1)
grid_search_catboost.fit(X, y)

# Print the results for each hyperparameter combination
print("\nCatBoost Hyperparameter Tuning Results:")
results_catboost = pd.DataFrame(grid_search_catboost.cv_results_)
print(results_catboost[['params', 'mean_test_score', 'std_test_score']])

# Get the best hyperparameters and score for CatBoost
best_params_catboost = grid_search_catboost.best_params_
best_score_catboost = grid_search_catboost.best_score_

print(f"\nBest Hyperparameters for CatBoost: {best_params_catboost}")
print(f"Best Mean ROC-AUC Score for CatBoost: {best_score_catboost:.4f}")


CatBoost Hyperparameter Tuning Results:
                                               params  mean_test_score  \
0   {'classifier__bagging_temperature': 0.0, 'clas...         0.788666   
1   {'classifier__bagging_temperature': 0.0, 'clas...         0.789551   
2   {'classifier__bagging_temperature': 0.0, 'clas...         0.802563   
3   {'classifier__bagging_temperature': 0.0, 'clas...         0.801909   
4   {'classifier__bagging_temperature': 0.0, 'clas...         0.794577   
5   {'classifier__bagging_temperature': 0.0, 'clas...         0.793403   
6   {'classifier__bagging_temperature': 0.0, 'clas...         0.803177   
7   {'classifier__bagging_temperature': 0.0, 'clas...         0.804307   
8   {'classifier__bagging_temperature': 0.0, 'clas...         0.793626   
9   {'classifier__bagging_temperature': 0.0, 'clas...         0.792958   
10  {'classifier__bagging_temperature': 0.0, 'clas...         0.804366   
11  {'classifier__bagging_temperature': 0.0, 'clas...         0.802596 

In [7]:
# Create a CatBoostClassifier with the best hyperparameters from the provided example
catboost_model = CatBoostClassifier(
    bagging_temperature=0.0,
    depth=6,
    iterations=100,
    learning_rate=0.1,
    random_strength=0.5,
    verbose=0,
    random_state=42
)

# Create the pipeline
catboost_pipeline = Pipeline([('preprocessor', preprocessor), ('classifier', catboost_model)])

# Train the CatBoost model using the best hyperparameters
catboost_pipeline.fit(X, y)

# Make predictions on the test set
predictions = catboost_pipeline.predict_proba(X_test)[:, 1]

binary_predictions = [1.0 if prob >= 0.5 else 0.0 for prob in predictions]  # Threshold at 0.5

submission = pd.DataFrame({
    'patient_id': test_data['patient_id'],
    'DiagPeriodL90D': binary_predictions
})

# Save the predictions to a CSV file
submission.to_csv('catboost_submission.csv', index=False)
print("CatBoost predictions saved to catboost_submission.csv")

CatBoost predictions saved to catboost_submission.csv
